### An attempt in creating in automated market scraper for all current closed and open markets
#### Multi-threaded event parser, (excluding MVE: sports parlays)

In [17]:
import requests

def get_clean_markets(status="open"):
    """Fetch single-leg markets only, excluding MVE combos."""
    base_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
    all_markets = []
    cursor = None
    
    while True:
        params = {
            "limit": 1000,
            "status": status,
            "mve_filter": "exclude"  # Filter out multivariate/parlay markets
        }
        if cursor:
            params["cursor"] = cursor
            
        response = requests.get(base_url, params=params)
        data = response.json()
        
        markets = data.get("markets", [])
        all_markets.extend(markets)
        
        cursor = data.get("cursor")
        if not cursor or not markets:
            break
    
    return all_markets

markets = get_clean_markets("open")

# Now titles should be clean
for m in markets[:1000]:
    print(f"{m['ticker']}: {m['title']}")

KXTABLETENNIS-26FEB111630BSUMPI-MPI: Will Milosz Piecowski win the Bartek Sulkowski vs Milosz Piecowski Table Tennis Match Winner?
KXTABLETENNIS-26FEB111630BSUMPI-BSU: Will Bartek Sulkowski win the Bartek Sulkowski vs Milosz Piecowski Table Tennis Match Winner?
KXPSAKIMENTION-26FEB12-VICT: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-TRUM: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-REDA: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-PAM: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-KASH: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-INVE: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-HOWA: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXPSAKIMENTION-26FEB12-GHIS: What will Jamie Raskin say during The Briefing With Jen Psaki?
KXP

In [11]:
"""
Kalshi Events Scraper
Pulls all open and closed events using multi-threading and saves to CSV.

Usage:
    python kalshi_scraper.py
    
Output:
    kalshi_events_YYYYMMDD_HHMMSS.csv
"""

import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2/events"

def get_session() -> requests.Session:
    """Create a session with retry logic."""
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("https://", adapter)
    return session

def fetch_events_page(session: requests.Session, status: str = None, cursor: str = None) -> dict:
    """Fetch a single page of events."""
    params = {"limit": 200}  # Events API max is likely 200, not 1000
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor
    
    response = session.get(BASE_URL, params=params, timeout=30)
    response.raise_for_status()
    return response.json()

def fetch_all_events_for_status(status: str = None) -> list:
    """Fetch all events for a given status (handles pagination). If status is None, fetches all."""
    session = get_session()
    all_events = []
    cursor = None
    page = 0
    label = status if status else "all"
    
    while True:
        page += 1
        data = fetch_events_page(session, status, cursor)
        events = data.get("events", [])
        
        if not events:
            break
            
        # Add status to each event for tracking
        for event in events:
            event["scrape_status"] = label
        
        all_events.extend(events)
        print(f"  [{label}] Page {page}: fetched {len(events)} events (total: {len(all_events)})")
        
        cursor = data.get("cursor")
        if not cursor:
            break
        
        # Small delay to be nice to the API
        time.sleep(0.1)
    
    return all_events

def scrape_all_events() -> list:
    """Scrape all events using multi-threading for different statuses."""
    statuses = ["open", "closed", "settled"]
    all_events = []
    
    print(f"Starting scrape at {datetime.now().isoformat()}")
    print("-" * 50)
    
    # First try with status filters in parallel
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_status = {
            executor.submit(fetch_all_events_for_status, status): status 
            for status in statuses
        }
        
        for future in as_completed(future_to_status):
            status = future_to_status[future]
            try:
                events = future.result()
                all_events.extend(events)
                print(f"  [{status}] Complete: {len(events)} events")
            except Exception as e:
                print(f"  [{status}] Error: {e}")
    
    # If status filters failed, try fetching all without filter
    if len(all_events) == 0:
        print("\nStatus filters failed. Trying without status filter...")
        try:
            all_events = fetch_all_events_for_status(None)
            print(f"  [all] Complete: {len(all_events)} events")
        except Exception as e:
            print(f"  [all] Error: {e}")
    
    print("-" * 50)
    print(f"Total events scraped: {len(all_events)}")
    
    return all_events

def events_to_csv(events: list, filename: str):
    """Save events to CSV file."""
    if not events:
        print("No events to save.")
        return
    
    # Define columns to extract
    columns = [
        "event_ticker",
        "series_ticker", 
        "title",
        "sub_title",
        "category",
        "mutually_exclusive",
        "available_on_brokers",
        "collateral_return_type",
        "strike_period",
        "scrape_status"
    ]
    
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=columns, extrasaction="ignore")
        writer.writeheader()
        writer.writerows(events)
    
    print(f"Saved {len(events)} events to {filename}")

def main():
    # Scrape all events
    events = scrape_all_events()
    
    # Save to CSV with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"kalshi_events_{timestamp}.csv"
    events_to_csv(events, filename)
    
    # Print summary by category
    print("\n" + "=" * 50)
    print("SUMMARY BY CATEGORY")
    print("=" * 50)
    
    categories = {}
    for event in events:
        cat = event.get("category", "Unknown")
        categories[cat] = categories.get(cat, 0) + 1
    
    for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
        print(f"  {cat}: {count}")
    
    # Print summary by status
    print("\n" + "=" * 50)
    print("SUMMARY BY STATUS")
    print("=" * 50)
    
    statuses = {}
    for event in events:
        status = event.get("scrape_status", "Unknown")
        statuses[status] = statuses.get(status, 0) + 1
    
    for status, count in sorted(statuses.items(), key=lambda x: -x[1]):
        print(f"  {status}: {count}")
    
    return filename

if __name__ == "__main__":
    output_file = main()
    print(f"\nDone! Output: {output_file}")

Starting scrape at 2026-02-10T19:54:30.253911
--------------------------------------------------
  [open] Page 1: fetched 200 events (total: 200)
  [settled] Page 1: fetched 200 events (total: 200)
  [closed] Page 1: fetched 200 events (total: 200)
  [open] Page 2: fetched 200 events (total: 400)
  [open] Page 3: fetched 200 events (total: 600)
  [settled] Page 2: fetched 200 events (total: 400)
  [closed] Page 2: fetched 200 events (total: 400)
  [settled] Page 3: fetched 200 events (total: 600)
  [closed] Page 3: fetched 53 events (total: 453)
  [closed] Complete: 453 events
  [settled] Page 4: fetched 200 events (total: 800)
  [settled] Page 5: fetched 200 events (total: 1000)
  [settled] Page 6: fetched 200 events (total: 1200)
  [settled] Page 7: fetched 200 events (total: 1400)
  [open] Page 4: fetched 200 events (total: 800)
  [settled] Page 8: fetched 200 events (total: 1600)
  [open] Page 5: fetched 200 events (total: 1000)
  [settled] Page 9: fetched 200 events (total: 1800)


In [15]:
"""
Kalshi Events Scraper
Pulls all open and closed events using multi-threading and saves to CSV.
Supports filtering by minimum volume (contracts) or dollar volume.

Usage:
    python kalshi_scraper.py                              # All events
    python kalshi_scraper.py --min-volume 1000            # Events with 1000+ contracts
    python kalshi_scraper.py --min-dollar-volume 1000000  # Events with $1M+ dollar volume
    python kalshi_scraper.py --min-dollar-volume 1000000 --status closed  # Closed $1M+ events
    
Output:
    kalshi_events_YYYYMMDD_HHMMSS.csv
    
Note:
    Dollar volume is estimated as sum(volume * last_price) across all markets.
    If no price data is available, assumes $0.50 per contract.
"""

import argparse
import requests
import csv
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2/events"

def get_session() -> requests.Session:
    """Create a session with retry logic."""
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount("https://", adapter)
    return session

def fetch_events_page(session: requests.Session, status: str = None, cursor: str = None) -> dict:
    """Fetch a single page of events with nested market data."""
    params = {
        "limit": 200,
        "with_nested_markets": "true"  # Include market data for volume calculation
    }
    if status:
        params["status"] = status
    if cursor:
        params["cursor"] = cursor
    
    response = session.get(BASE_URL, params=params, timeout=30)
    response.raise_for_status()
    return response.json()

def calculate_event_volume(event: dict) -> int:
    """Calculate total volume (contracts) across all markets in an event."""
    markets = event.get("markets", [])
    return sum(m.get("volume", 0) for m in markets)

def calculate_event_dollar_volume(event: dict) -> float:
    """
    Estimate dollar volume across all markets in an event.
    
    Dollar volume = sum of (volume * last_price) for each market.
    Prices are in cents, so we divide by 100 to get dollars.
    
    Falls back to volume * 0.50 if no price data available.
    """
    markets = event.get("markets", [])
    total_dollar_volume = 0.0
    
    for m in markets:
        volume = m.get("volume", 0)
        # last_price is in cents (0-100)
        last_price = m.get("last_price", 50)  # Default to 50 cents if no price
        if last_price == 0:
            last_price = 50  # Assume 50 cents for markets with no trades
        
        # Convert cents to dollars and multiply by volume
        dollar_volume = volume * (last_price / 100.0)
        total_dollar_volume += dollar_volume
    
    return total_dollar_volume

def calculate_event_open_interest(event: dict) -> int:
    """Calculate total open interest across all markets in an event."""
    markets = event.get("markets", [])
    return sum(m.get("open_interest", 0) for m in markets)

def get_event_close_time(event: dict) -> str:
    """Get the latest close time from markets in an event."""
    markets = event.get("markets", [])
    close_times = [m.get("close_time", "") for m in markets if m.get("close_time")]
    return max(close_times) if close_times else ""

def get_market_count(event: dict) -> int:
    """Get the number of markets in an event."""
    return len(event.get("markets", []))

def fetch_all_events_for_status(status: str = None, min_volume: int = 0, min_dollar_volume: float = 0) -> list:
    """Fetch all events for a given status (handles pagination). If status is None, fetches all."""
    session = get_session()
    all_events = []
    cursor = None
    page = 0
    label = status if status else "all"
    filtered_count = 0
    
    while True:
        page += 1
        data = fetch_events_page(session, status, cursor)
        events = data.get("events", [])
        
        if not events:
            break
        
        for event in events:
            # Calculate aggregate stats from nested markets
            total_volume = calculate_event_volume(event)
            total_dollar_volume = calculate_event_dollar_volume(event)
            
            # Apply volume filters
            if total_volume < min_volume:
                filtered_count += 1
                continue
            if total_dollar_volume < min_dollar_volume:
                filtered_count += 1
                continue
            
            # Enrich event with computed fields
            event["scrape_status"] = label
            event["total_volume"] = total_volume
            event["total_dollar_volume"] = round(total_dollar_volume, 2)
            event["total_open_interest"] = calculate_event_open_interest(event)
            event["close_time"] = get_event_close_time(event)
            event["market_count"] = get_market_count(event)
            
            all_events.append(event)
        
        print(f"  [{label}] Page {page}: processed {len(events)} events, kept {len(all_events)} (filtered {filtered_count})")
        
        cursor = data.get("cursor")
        if not cursor:
            break
        
        # Small delay to be nice to the API
        time.sleep(0.1)
    
    return all_events

def scrape_all_events(statuses: list, min_volume: int = 0, min_dollar_volume: float = 0) -> list:
    """Scrape all events using multi-threading for different statuses."""
    all_events = []
    
    print(f"Starting scrape at {datetime.now().isoformat()}")
    print(f"Filters: statuses={statuses}, min_volume={min_volume:,}, min_dollar_volume=${min_dollar_volume:,.0f}")
    print("-" * 60)
    
    # Fetch different statuses in parallel
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_status = {
            executor.submit(fetch_all_events_for_status, status, min_volume, min_dollar_volume): status 
            for status in statuses
        }
        
        for future in as_completed(future_to_status):
            status = future_to_status[future]
            try:
                events = future.result()
                all_events.extend(events)
                print(f"  [{status}] Complete: {len(events)} events")
            except Exception as e:
                print(f"  [{status}] Error: {e}")
    
    # If status filters failed, try fetching all without filter
    if len(all_events) == 0:
        print("\nStatus filters failed. Trying without status filter...")
        try:
            all_events = fetch_all_events_for_status(None, min_volume, min_dollar_volume)
            print(f"  [all] Complete: {len(all_events)} events")
        except Exception as e:
            print(f"  [all] Error: {e}")
    
    print("-" * 60)
    print(f"Total events scraped: {len(all_events)}")
    
    return all_events

def events_to_csv(events: list, filename: str):
    """Save events to CSV file."""
    if not events:
        print("No events to save.")
        return
    
    # Define columns to extract (including computed fields)
    columns = [
        "event_ticker",
        "series_ticker", 
        "title",
        "sub_title",
        "category",
        "mutually_exclusive",
        "total_volume",
        "total_dollar_volume",
        "total_open_interest",
        "market_count",
        "close_time",
        "scrape_status",
        "available_on_brokers",
        "collateral_return_type",
        "strike_period",
    ]
    
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=columns, extrasaction="ignore")
        writer.writeheader()
        writer.writerows(events)
    
    print(f"Saved {len(events)} events to {filename}")

def main():
    # Parse command line arguments
    parser = argparse.ArgumentParser(description="Scrape Kalshi events to CSV")
    parser.add_argument(
        "--min-volume", 
        type=int, 
        default=0,
        help="Minimum total contracts traded across all markets in an event (default: 0)"
    )
    parser.add_argument(
        "--min-dollar-volume",
        type=float,
        default=0,
        help="Minimum estimated dollar volume in USD (default: 0). Use 1000000 for $1M+"
    )
    parser.add_argument(
        "--status",
        type=str,
        nargs="+",
        default=["open", "closed", "settled"],
        choices=["open", "closed", "settled"],
        help="Event statuses to fetch (default: open closed settled)"
    )
    parser.add_argument(
        "--output",
        type=str,
        default=None,
        help="Output filename (default: kalshi_events_TIMESTAMP.csv)"
    )
    args = parser.parse_args()
    
    # Scrape all events
    events = scrape_all_events(args.status, args.min_volume, args.min_dollar_volume)
    
    # Save to CSV with timestamp
    if args.output:
        filename = args.output
    else:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"kalshi_events_{timestamp}.csv"
    
    events_to_csv(events, filename)
    
    # Print summary by category
    print("\n" + "=" * 60)
    print("SUMMARY BY CATEGORY")
    print("=" * 60)
    
    categories = {}
    for event in events:
        cat = event.get("category", "Unknown")
        categories[cat] = categories.get(cat, 0) + 1
    
    for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
        print(f"  {cat}: {count}")
    
    # Print summary by status
    print("\n" + "=" * 60)
    print("SUMMARY BY STATUS")
    print("=" * 60)
    
    statuses = {}
    for event in events:
        status = event.get("scrape_status", "Unknown")
        statuses[status] = statuses.get(status, 0) + 1
    
    for status, count in sorted(statuses.items(), key=lambda x: -x[1]):
        print(f"  {status}: {count}")
    
    # Print volume stats
    if events:
        volumes = [e.get("total_volume", 0) for e in events]
        dollar_volumes = [e.get("total_dollar_volume", 0) for e in events]
        print("\n" + "=" * 60)
        print("VOLUME STATISTICS")
        print("=" * 60)
        print(f"  Total contracts: {sum(volumes):,}")
        print(f"  Total dollar volume: ${sum(dollar_volumes):,.2f}")
        print(f"  Mean dollar volume: ${sum(dollar_volumes) / len(dollar_volumes):,.2f}")
        print(f"  Max dollar volume: ${max(dollar_volumes):,.2f}")
        print(f"  Min dollar volume: ${min(dollar_volumes):,.2f}")
    
    return filename

if __name__ == "__main__":
    output_file = main()
    print(f"\nDone! Output: {output_file}")

usage: ipykernel_launcher.py [-h] [--min-volume MIN_VOLUME]
                             [--min-dollar-volume MIN_DOLLAR_VOLUME]
                             [--status {open,closed,settled} [{open,closed,settled} ...]]
                             [--output OUTPUT]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/yiyangyu/Library/Jupyter/runtime/kernel-v36cd99daedee8a7535c8edd4f46c2843d017073fb.json


SystemExit: 2

In [16]:
# Add this temporarily to see all fields in a market
import requests
url = "https://api.elections.kalshi.com/trade-api/v2/events?limit=1&with_nested_markets=true"
response = requests.get(url)
data = response.json()
if data.get("events"):
    market = data["events"][0].get("markets", [{}])[0]
    print("Available market fields:")
    for key in sorted(market.keys()):
        print(f"  {key}: {market.get(key)}")

Available market fields:
  can_close_early: True
  close_time: 2099-08-01T04:59:00Z
  created_time: 0001-01-01T00:00:00Z
  early_close_condition: This market will close and expire early if the event occurs.
  event_ticker: KXELONMARS-99
  expected_expiration_time: 2099-08-01T15:00:00Z
  expiration_time: 2099-08-08T15:00:00Z
  expiration_value: 
  last_price: 8
  last_price_dollars: 0.0800
  latest_expiration_time: 2099-08-08T15:00:00Z
  liquidity: 5883237
  liquidity_dollars: 58832.3700
  market_type: binary
  no_ask: 93
  no_ask_dollars: 0.9300
  no_bid: 92
  no_bid_dollars: 0.9200
  no_sub_title: Mars
  notional_value: 100
  notional_value_dollars: 1.0000
  open_interest: 15115
  open_interest_fp: 15115.00
  open_time: 2025-08-28T20:45:00Z
  previous_price: 0
  previous_price_dollars: 0.0000
  previous_yes_ask: 0
  previous_yes_ask_dollars: 0.0000
  previous_yes_bid: 0
  previous_yes_bid_dollars: 0.0000
  price_level_structure: linear_cent
  price_ranges: [{'end': '1.0000', 'start': 

In [19]:
import requests

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2/events"

for status in ["open", "closed", "settled", None]:
    params = {"limit": 200, "with_nested_markets": "true"}
    if status:
        params["status"] = status
    
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    events = data.get("events", [])
    cursor = data.get("cursor", "")
    
    label = status if status else "no filter"
    print(f"{label}: {len(events)} events on first page, has_more={bool(cursor)}")

open: 200 events on first page, has_more=True
closed: 200 events on first page, has_more=True
settled: 200 events on first page, has_more=True
no filter: 200 events on first page, has_more=True


In [24]:
import requests

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2/events"

# Fetch a few pages of closed events and check their volumes
all_volumes = []
cursor = None

for page in range(10):  # Fetch 10 pages = 2000 events
    params = {"limit": 200, "status": "closed", "with_nested_markets": "true"}
    if cursor:
        params["cursor"] = cursor
    
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    
    for event in data.get("events", []):
        markets = event.get("markets", [])
        total_vol = sum(m.get("volume", 0) for m in markets)
        all_volumes.append((event.get("title", "")[:50], total_vol))
    
    cursor = data.get("cursor")
    if not cursor:
        break

# Sort by volume and show top 20
all_volumes.sort(key=lambda x: -x[1])

print(f"Total events checked: {len(all_volumes)}")
print(f"\nTop 20 by dollar volume:")
for i, (title, vol) in enumerate(all_volumes[:20]):
    print(f"  {i+1}. ${vol:>12,}  {title}")

print(f"\nEvents with $1M+: {sum(1 for _, v in all_volumes if v >= 1_000_000)}")
print(f"Events with $500K+: {sum(1 for _, v in all_volumes if v >= 500_000)}")
print(f"Events with $100K+: {sum(1 for _, v in all_volumes if v >= 100_000)}")

Total events checked: 466

Top 20 by dollar volume:
  1. $   4,780,801  Recession in 2025?
  2. $     237,398  S&P price on Feb 11, 2026 at 4pm EST?
  3. $     187,097  Highest temperature in Washington DC on Feb 5, 202
  4. $     175,675  What will be Odell Beckham Jr.'s next team?
  5. $     157,952  How much revenue will tariffs bring in in 2025?
  6. $     116,249  SpaceX Starship launch?
  7. $      84,146  What will the announcers say during the San Antoni
  8. $      76,673  January 2026 CPI Combo: CPI & CPI (YoY)
  9. $      31,056  Who will be the next head coach of New York Pro Me
  10. $      30,264  S&P price range on Feb 11, 2026 at 4pm EST?
  11. $      24,701  When will OpenSea launch a token?
  12. $      24,366  Will another GOP Member of Congress call for Noem'
  13. $      20,356  How many guinea worm cases will there be this year
  14. $      20,161  Nasdaq-100 price on Feb 11, 2026 at 4pm EST?
  15. $      16,364  How much will core PCE increase in Dec 2025?
  16. 

In [25]:
import requests

BASE_URL = "https://api.elections.kalshi.com/trade-api/v2/events"

for status in ["open", "closed", "settled"]:
    all_volumes = []
    cursor = None
    
    while True:
        params = {"limit": 200, "status": status, "with_nested_markets": "true"}
        if cursor:
            params["cursor"] = cursor
        
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        events = data.get("events", [])
        
        if not events:
            break
        
        for event in events:
            markets = event.get("markets", [])
            total_vol = sum(m.get("volume", 0) for m in markets)
            all_volumes.append((event.get("title", "")[:40], total_vol))
        
        cursor = data.get("cursor")
        if not cursor:
            break
    
    # Stats
    over_1m = sum(1 for _, v in all_volumes if v >= 1_000_000)
    over_100k = sum(1 for _, v in all_volumes if v >= 100_000)
    
    print(f"{status:8}: {len(all_volumes):,} events | $1M+: {over_1m} | $100K+: {over_100k}")
    
    # Show top 3 for this status
    all_volumes.sort(key=lambda x: -x[1])
    for title, vol in all_volumes[:3]:
        print(f"          ${vol:>12,}  {title}")
    print()

open    : 4,196 events | $1M+: 88 | $100K+: 432
          $ 159,220,951  Who will Trump nominate as Fed Chair?
          $  47,147,405  2028 Democratic nominee for President?
          $  25,457,230  When will Bitcoin hit $150k?

closed  : 474 events | $1M+: 1 | $100K+: 9
          $   4,780,801  Recession in 2025?
          $     305,550  Hapoel Jerusalem vs Reyer Venezia
          $     237,398  S&P price on Feb 11, 2026 at 4pm EST?



KeyboardInterrupt: 